In [1]:
from bs4 import BeautifulSoup
import re
import requests
import time
import json
import lxml
import urllib3
import pandas as pd
urllib3.disable_warnings()

5 колонок:
1. Полное наименование/ФИО
2. ОГРН
3. ИНН
4. Дата принятия решения о включении
5. Дата принятия решения об исключении

Тэги для колонок:
1. Наименование: 'th', 'data-column' = '0'
2. ОГРН: 'th', 'data-column' = '1'
3. ИНН: 'th', 'data-column' = '2'
4. Дата принятия решения о включении: 'th', 'data-column' = '3'
5. Дата принятия решения об исключении: 'th', 'data-column' = '4'

Тэги для элементов таблицы:

Общий тэг: 'tr', 'role' = 'row'
Для именования: 'td' (надо отметить, что собираю текст)
ОГРН: 'td', 'data-text' = '...'(13 символов)
ИНН: 'td', 'data-text' = '...'(10 символов, ИНН только у организаций)
Включение в реестр: 'td'
Исключение из реестра: 'td'

In [2]:
# Соберу сюда, чтобы не отправлять множественные запросы снова и снова на сайт
url = 'https://gogov.ru/articles/inagenty-21apr22'

headers = {'User-Agent': 'Mozilla/5.0 (Macintosh; Intel Mac OS X 10_10_1) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/39.0.2171.95 Safari/537.36'} # не мой хедер
request = requests.get(url, headers = headers)
print(request.content)

b'<!DOCTYPE html>\r\n<html lang="ru-RU">\r\n<head>\r\n\t<title>\xd0\xa0\xd0\xb5\xd0\xb5\xd1\x81\xd1\x82\xd1\x80 \xd0\xb8\xd0\xbd\xd0\xbe\xd1\x81\xd1\x82\xd1\x80\xd0\xb0\xd0\xbd\xd0\xbd\xd1\x8b\xd1\x85 \xd0\xb0\xd0\xb3\xd0\xb5\xd0\xbd\xd1\x82\xd0\xbe\xd0\xb2 \xd0\xb2 \xd0\xa0\xd0\xbe\xd1\x81\xd1\x81\xd0\xb8\xd0\xb8: \xd0\xbf\xd0\xbe\xd0\xbb\xd0\xbd\xd1\x8b\xd0\xb9 \xd1\x81\xd0\xbf\xd0\xb8\xd1\x81\xd0\xbe\xd0\xba \xd0\xbd\xd0\xb0 1 \xd0\xb4\xd0\xb5\xd0\xba\xd0\xb0\xd0\xb1\xd1\x80\xd1\x8f 2023 \xd0\xb3\xd0\xbe\xd0\xb4\xd0\xb0</title>\r\n\t<meta http-equiv="Content-Type" content="text/html; charset=utf-8" />\r\n\t<meta name="viewport" content="width=device-width, initial-scale=1, maximum-scale=1">\r\n\t<meta property="og:image" content="https://gogov.ru/img/884064.jpg" />\r\n\t<link rel="stylesheet" href="/wp-content/themes/theme/style.css?v=3.0.1" />\r\n\t<link rel="icon" type="image/svg+xml" href="//i.gogov.ru/favicon.svg">\r\n\t<link rel="canonical" href="https://gogov.ru/articles/inage

In [3]:
name_list = []

In [4]:
soup = BeautifulSoup(request.text, 'lxml')

In [5]:
# letters_pattern =  r'\b([а-я]*[А-Я][а-я]*)\b' 

for n in soup.find_all('td'):
    name = n.get_text()
    name_list.append(name)
print(len(name_list))

# Разделить, чтобы каждое шестое значение попадало в нужный список


3770


В итоге сделаю по-другому: сделаю срезы по нужным элементам списка, выделю их в новые списки с сформирую датафрейм

In [6]:
slice_name = slice(0, 3770, 5)
new_name_list = name_list[slice_name]
print(new_name_list)

['«Евразийская антимонопольная ассоциация»', 'Ассоциация некоммерческих организаций «В защиту прав избирателей «ГОЛОС»', 'Региональная общественная\r\nправозащитная организация «Союз\r\n«Женщины Дона»', 'Автономная некоммерческая научно- исследовательская организация\r\n«Центр социальной политики и гендерных исследований»', 'Региональная общественная организация в защиту демократических прав и свобод\r\n«ГОЛОС»', 'Некоммерческая организация Фонд\r\n«Костромской центр поддержки общественных инициатив»', 'Калининградская региональная общественная организация\r\n«Экозащита!-Женсовет»', 'Фонд содействия защите прав и свобод граждан «Общественный вердикт»', 'Межрегиональная общественная организация Правозащитный Центр\r\n«Мемориал»', 'Автономная некоммерческая организация «Юристы за конституционные права и свободы»', 'Межрегиональная Ассоциация правозащитных общественных объединений «Правозащитная ассоциация»', 'Санкт-Петербургская региональная общественная правозащитная организация «Солдат

In [7]:
slice_name = slice(1, 3770, 5)
new_ogrn_list = name_list[slice_name]
print(new_ogrn_list)

['1097799044518', '1037739618872', '1026100023872', '1036405207640', '1067799008860', '1054408732256', '1093900001689', '1047796086205', '1027739615860', '1037709052028', '1051664047984', '1037858013093', '1087800001486', '1037739320431', '1026402679148', '1037700033216', '1037867003470', '1037739960554', '1072900001448', '1090900000300', '1037739134432', '1022300002867', '1053918505475', '1037700082177', '1037858000070', '1127799003683', '1027800003758', '1087800000738', '1021200003857', '1026000971590', '1026600008082', '1025200024035', '1027809252811', '1113400000658', '1027900000963', '1035401487890', '1095100000896', '1076300000809', '1035400002197', '1075900006600', '1026300005962', '1023601544449', '1027400000430', '1117400006460', '1025300006071', '1146300000197', '1035100261679', '1025100835429', '1046300561273', '1097799002696', '1026100008802', '1027739847981', '1027400012210', '1085400002742', '1027739104293', '1117799004740', '1033918501715', '1025900529005', '106100105318

In [8]:
slice_name = slice(2, 3770, 5)
new_inn_list = name_list[slice_name]
print(new_inn_list)

['7729441207', '7702295527', '6150025245', '6452083317', '7709439312', '4401059496', '3907062893', '7708516356', '7707061466', '7709427476', '1655065434', '7809026358', '7840018077', '7729348818', '6452043709', '7707304743', '7841002055', '7709515010', '2901168740', '917014100', '7702240006', '2311056659', '3908032115', '7709051150', '7826018607', '7703479573', '7825059110', '7801268620', '1215063179', '6027043010', '6671126309', '5262090699', '7825403980', '3442112820', '7901018694', '5404145160', '5190909055', '6317067380', '5408166026', '5902212468', '6316079826', '3666064690', '7448037969', '7453990422', '5321053490', '6311998417', '5190119160', '5190109041', '6316093281', '7719287933', '6165084780', '7709311560', '7422025863', '5407053291', '7708037113', '7704300050', '3906072338', '5902188906', '1001015210', '3906187071', '3906159518', '7804048809', '6660044647', '6661052880', '5260247111', '7710251587', '7704224930', '7113021357', '411153758', '5404138194', '1001028360', '263409

In [9]:
slice_name = slice(3, 3770, 5)
new_registry_on_list = name_list[slice_name]
print(new_registry_on_list)

['27.06.2013', '05.06.2014', '05.06.2014', '05.06.2014', '05.06.2014', '05.06.2014', '21.07.2014', '21.07.2014', '21.07.2014', '21.07.2014', '21.07.2014', '28.08.2014', '28.08.2014', '03.09.2014', '02.10.2014', '20.11.2014', '20.11.2014', '09.12.2014', '15.12.2014', '15.12.2014', '22.12.2014', '25.12.2014', '25.12.2014', '25.12.2014', '30.12.2014', '30.12.2014', '30.12.2014', '30.12.2014', '30.12.2014', '30.12.2014', '16.01.2015', '16.01.2015', '20.01.2015', '20.01.2015', '26.01.2015', '30.01.2015', '04.02.2015', '06.02.2015', '12.02.2015', '13.02.2015', '16.02.2015', '26.02.2015', '06.03.2015', '06.03.2015', '06.03.2015', '13.03.2015', '13.03.2015', '19.03.2015', '20.03.2015', '27.03.2015', '03.04.2015', '07.04.2015', '15.04.2015', '17.04.2015', '20.04.2015', '24.04.2015', '29.04.2015', '29.04.2015', '07.05.2015', '13.05.2015', '13.05.2015', '13.05.2015', '15.05.2015', '15.05.2015', '22.05.2015', '25.05.2015', '25.05.2015', '28.05.2015', '10.06.2015', '19.06.2015', '19.06.2015', '19.0

In [10]:
slice_name = slice(4, 3771, 5)
new_registry_off_list = name_list[slice_name]
print(new_registry_off_list)

['-', '13.03.2020', '29.02.2016', '22.05.2015', '20.02.2017', '19.06.2015', '-', '-', '26.05.2022', '26.05.2015', '29.12.2016', '23.10.2015', '-', '24.02.2016', '06.11.2015', '-', '25.03.2022', '17.03.2021', '12.09.2019', '05.04.2018', '30.12.2015', '22.04.2016', '21.07.2021', '-', '22.05.2015', '-', '-', '03.11.2015', '-', '12.04.2017', '-', '13.09.2016', '-', '22.07.2015', '22.05.2015', '16.01.2024', '28.10.2015', '25.04.2019', '30.01.2018', '19.06.2015', '24.01.2020', '-', '02.05.2017', '17.05.2017', '31.05.2017', '22.07.2016', '25.08.2015', '16.10.2015', '08.10.2015', '25.11.2016', '30.03.2016', '-', '09.10.2018', '12.05.2016', '-', '11.07.2017', '16.12.2015', '18.08.2016', '25.08.2015', '30.03.2016', '-', '25.11.2022', '-', '-', '31.08.2017', '27.05.2020', '11.09.2015', '13.12.2016', '11.12.2017', '21.09.2016', '25.03.2016', '11.09.2018', '-', '21.06.2016', '07.10.2019', '27.01.2017', '25.01.2019', '01.12.2015', '24.12.2019', '28.03.2016', '02.08.2021', '20.01.2020', '28.10.2016',

In [11]:
# df = pd.DataFrame(columns = ['Name', 'OGRN', 'INN', 'Registry_on', 'Registry_off'])
df = pd.DataFrame(({'Name': new_name_list, 'ORGN': new_ogrn_list, 'INN' : new_inn_list, 'Registry_on' : new_registry_on_list, 'Registry_off' : new_registry_off_list}))
df

,Name,ORGN,INN,Registry_on,Registry_off
0,«Евразийская антимонопольная ассоциация»,1097799044518,7729441207,27.06.2013,-
1,Ассоциация некоммерческих организаций «В защит...,1037739618872,7702295527,05.06.2014,13.03.2020
2,Региональная общественная\r\nправозащитная орг...,1026100023872,6150025245,05.06.2014,29.02.2016
3,Автономная некоммерческая научно- исследовател...,1036405207640,6452083317,05.06.2014,22.05.2015
4,Региональная общественная организация в защиту...,1067799008860,7709439312,05.06.2014,20.02.2017
...,...,...,...,...,...
749,Чирикова Евгения Сергеевна,-,771304157658,19.01.2024,-
750,Живица Владислав Геннадьевич,-,732898815791,19.01.2024,-
751,Олейник Владислав Александрович,-,780627729699,19.01.2024,-
752,Лекторий «Живое слово»,-,-,19.01.2024,-
